### TODO
- See how we can jit stuff here

In [1]:
import numpy as np
import scipy as sp
import sys
sys.path.append('C:/Users/rannu/OneDrive - NTNU/Desktop/VsPython/'+\
                'Spain/NMfDS/Assignments/Ass6')
from custom_ODE_solver import ODE_solver
sys.path.append('C:/Users/rannu/OneDrive - NTNU/Desktop/VsPython/'+\
                'Spain/NMfDS/Assignments/Ass4')
from RTBP_definitions import r1, r2, OMEGA, ODE_R3BP, Jacobi_first_integral


In [2]:
X0 = np.array([1.119748482176185, 0,0.,-0.2260036463376957], 
            dtype=np.float64)
t_max = np.float64(6.219656012174353)
eval_pts = 1000 # Integer
mu =np.float64(0.012)

sol = ODE_solver(lambda t, X: ODE_R3BP(t, mu, X), X0, t_max, eval_pts, 
                 hamiltonian=lambda X: Jacobi_first_integral(mu, X[0], X[1], 
                                                             X[2], X[3]))

# Final values of the solution
print("Final values of the solution:")
print("x = ", sol.y[:, -1])

Final values of the solution:
x =  [ 1.11974848e+00  5.06660686e-13  2.31125882e-13 -2.26003646e-01]


c:\Users\rannu\OneDrive - NTNU\Desktop\VsPython\ntnusession\lib\site-packages\scipy\integrate\_ivp\common.py:47: UserWarning: At least one element of `rtol` is too small. Setting `rtol = np.maximum(rtol, 2.220446049250313e-14)`.
  warn("At least one element of `rtol` is too small. "


In [38]:
# Rewrite poincare_map_solve_ivp to work for RTBP defined above, 
# using the function ODE_solver
def poincare_map_solve_ivp_R3BP(ODE_R3BP, initial_conditions, dir, 
                                step, t_span, mu):
    # Procedure to compute when the x-axis is crossed
    # for the first time
    product = 1
    time = 0
    startPoint = np.array(initial_conditions)
    initial_conditions = initial_conditions

    while product >= 0 and abs(time) < abs(step*100):
        solution = ODE_solver(ODE_R3BP, startPoint, t_span[1], 1000, 
                    t_min=t_span[0],
                    hamiltonian=lambda X: Jacobi_first_integral(mu, X[0], X[1], 
                                                             X[2], X[3]))
        Y = solution.y.T  # Transposing to match previous structure
        product = Y[1, 1] * Y[-1, 1] # Check if x-axis is crossed
        startPoint = Y[-1, :]
        t_span = [t_span[0]  + dir * step, t_span[1] + dir * step]
        time += step*dir
    # make an error if the x-axis is never crossed
    if abs(time) >= abs(step*100):
        raise ValueError("No crossing found" + "\ntime:  " + str(time) + \
                                "\nproduct  " + str(product))
    
    # Procedure to compute the exact time of the crossing
    for i in range(100):
        solution = ODE_solver(ODE_R3BP, initial_conditions, time, 1000, 
                 hamiltonian=lambda X: Jacobi_first_integral(mu, X[0], X[1], 
                                                             X[2], X[3]))
        Y = solution.y.T
        # One iteration of Newton's method.
        difference = Y[-1, 1] / ODE_R3BP(0, Y[-1, :])[1]
        time -= difference
        # Make a debug printout for the above part of the loop
        # print("\nIteration", i)
        # print("Y[-1, 1]  ", Y[-1, 1])
        # print("ODE_R3BP(0, Y[-1, :])[1]  ", ODE_R3BP(0, Y[-1, :])[1])
        # print("Y[-1, :]  ", Y[-1, :])
        # print("initial_conditions  ", initial_conditions)
        # print("time:  ", time, "\ndifference  ", difference)
        if i == 99:
            raise ValueError("No convergence" + "\ntime:  " + str(time) + \
                              "\ndifference  " + str(difference))
        if abs(difference) < 1e-15:
            #print("Convergence after", i, "iterations")
            break

    TimeDuration = time
    newInitial = Y[-1, :]
    return newInitial, TimeDuration

In [43]:
# Test poincare_map_solve_ivp_R3BP
initial_conditions = np.array([1.119748482176185, 0,0.,-0.2260036463376957])
dir = -1
step = 0.1
t_span = [0, step] # 6.219656012174353
mu = 0.012         
newInitial, TimeDuration1 = poincare_map_solve_ivp_R3BP(\
                                lambda t, X: ODE_R3BP(t, mu, X), 
                                initial_conditions, dir, step, t_span, mu)
print("newInitial = ", newInitial)
print("TimeDuration = ", TimeDuration1)

newInitial =  [8.90134016e-01 3.64291930e-17 4.23164108e-14 2.39581331e-01]
TimeDuration =  -3.1098280060876102


In [44]:
newInitial, TimeDuration2 = poincare_map_solve_ivp_R3BP(lambda t, X: ODE_R3BP(t, mu, X), 
                                                       newInitial, 
                                                       dir, step, t_span, mu)
print("newInitial = ", newInitial)
print("TimeDuration = ", TimeDuration1 + TimeDuration2)
#             6.219656012174353

newInitial =  [ 1.11974848e+00  2.42861287e-17  8.40508219e-14 -2.26003646e-01]
TimeDuration =  -6.219656012176599


### NUMBA speedup tests

In [6]:
import numba as nb

In [7]:
X0 = np.array([1.119748482176185, 0,0.,-0.2260036463376957], 
            dtype=np.float64)
t_max = np.float64(6.219656012174353)
eval_pts = 1000 # Integer
mu =np.float64(0.012)
dir = -1

sol = ODE_solver(lambda t, X: ODE_R3BP(t, mu, X), X0, t_max, eval_pts, 
                 hamiltonian=lambda X: Jacobi_first_integral(mu, X[0], X[1], 
                                                             X[2], X[3]))

Y = sol.y.T

def compute_product1(Y):
    return Y[1, 1] * Y[-1, 1]


@nb.jit
def compute_product2(Y):
    return Y[1, 1] * Y[-1, 1]

# Final values of the solution
# print("Final values of the solution:")
# print("x = ", sol.y[:, -1])

In [8]:

print("Run times INCLUDING compilation\n")
print("Slow code:")
#   % is IPython magic, invalid python code. to do the same thing, us the 
#   two lines below it...
#%timeit go_fast_1(x)
from IPython import get_ipython
get_ipython().magic('timeit compute_product1(Y)')

print("\nFast code:")
get_ipython().magic('timeit -n 1000 compute_product2(Y)')

print("\nRun times EXCLUDING compilation\n")

print("Slow code:")
get_ipython().magic('timeit compute_product1(Y)')

print("\nFast code:")
get_ipython().magic('timeit compute_product2(Y)')

Run times INCLUDING compilation

Slow code:


C:\Users\rannu\AppData\Local\Temp\ipykernel_4168\2923573487.py:7: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('timeit compute_product1(Y)')


274 ns ± 8.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)

Fast code:


C:\Users\rannu\AppData\Local\Temp\ipykernel_4168\2923573487.py:10: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('timeit -n 1000 compute_product2(Y)')


The slowest run took 3070.71 times longer than the fastest. This could mean that an intermediate result is being cached.
107 µs ± 261 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Run times EXCLUDING compilation

Slow code:


C:\Users\rannu\AppData\Local\Temp\ipykernel_4168\2923573487.py:15: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('timeit compute_product1(Y)')


273 ns ± 8.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)

Fast code:


C:\Users\rannu\AppData\Local\Temp\ipykernel_4168\2923573487.py:18: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('timeit compute_product2(Y)')


245 ns ± 3.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
